In [50]:
import pickle
import numpy as np
import iisignature

from tqdm import tqdm_notebook as tqdm

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import Pipeline

import sys
sys.path.append('../')

import warnings
warnings.filterwarnings('ignore')

from utils.addtime import AddTime, LeadLag

In [76]:
input1 = pickle.load(open('../data/crops/clim_bags_1.obj', 'rb'))
input2 = pickle.load(open('../data/crops/clim_bags_2.obj', 'rb'))
input_ = input1 + input2

targets = pickle.load(open('../data/crops/clim_labels.obj', 'rb'))

In [89]:
depth = 2

X = []

for bag in tqdm(input_):
    bag = LeadLag([0,1]).fit_transform(bag)
    bag = AddTime().fit_transform(bag)
    X.append(iisignature.sig(bag, depth).mean(0))

In [90]:
tuned_parameters = [{'svm__kernel': ['poly'], 'svm__gamma': ['auto'], 
                     'svm__C': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3], 'svm__degree': [1, 2, 3, 4, 5, 6]},
                    {'svm__kernel': ['rbf'], 'svm__gamma': ['auto'], 
                     'svm__C': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3]}]

In [91]:
pipe = Pipeline([('std_scaler', StandardScaler()), ('svm', SVR())])
clf = GridSearchCV(pipe, tuned_parameters, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
clf.fit(np.array(X), np.array(list(targets.values())))

Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    1.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('std_scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('svm',
                                        SVR(C=1.0, cache_size=200, coef0=0.0,
                                            degree=3, epsilon=0.1,
                                            gamma='scale', kernel='rbf',
                                            max_iter=-1, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'svm__C': [0.001, 0.01, 0.1, 1, 10.0, 100.0, 1000.0],
                          'svm__degree': [1, 2, 3, 4, 5, 6],
           

In [92]:
score = cross_val_score(clf.best_estimator_, np.array(X), np.array(list(targets.values())), scoring='neg_mean_squared_error')
print(-score.mean(), score.std())

0.7999141301746443 0.18368153540391047


In [88]:
clf.best_estimator_

Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm',
                 SVR(C=10.0, cache_size=200, coef0=0.0, degree=1, epsilon=0.1,
                     gamma='auto', kernel='poly', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)